## Pregunta 4 – Extensión 3 (2,0 puntos).

Ahora, DataMind Labs desea optimizar su red considerando transferencias intermedias y
restricciones adicionales:

* Se habilita un nodo de transbordo (Zúrich (T)), que puede recibir datos desde los orígenes y
reenviarlos a los destinos. El coste variable de transferencia es de 5 céntimos € por MilGb de
datos entre cada origen y Zúrich, y entre Zúrich y cada destino. Estos canales también
consumen capacidad y pagan coste fijo de activación (50 € o 65 € si es el quinto canal). Añade
las nuevas variables de flujo y de activación de canal correspondientes, y las restricciones de
balance de flujo en Zúrich en el modelo de la pregunta 3, para captar este supuesto.
* Añade la condición de que, si el canal de Zúrich a París se activa, debe transportar al menos
0,5 MilGb de datos, para captar este supuesto.

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# --- 1. DATOS DEL PROBLEMA ---

# Conjuntos de Nodos
Origenes = ['Lisboa', 'Madrid', 'Turin']
Destinos = ['Paris', 'Berlin', 'Varsovia']
Transbordo = 'Zurich'

# Rutas posibles
Rutas_Directas = gp.tuplelist([(i, j) for i in Origenes for j in Destinos])
Rutas_A_T = gp.tuplelist([(i, Transbordo) for i in Origenes])
Rutas_Desde_T = gp.tuplelist([(Transbordo, j) for j in Destinos])
Todas_Rutas = Rutas_Directas + Rutas_A_T + Rutas_Desde_T

# Oferta/Capacidad máxima de cada origen (S_i, en MilGb)
Oferta = {
    'Lisboa': 5,
    'Madrid': 6,
    'Turin': 7
}

# Demanda de cada destino (D_j, en MilGb)
Demanda = {
    'Paris': 4,
    'Berlin': 5,
    'Varsovia': 9
}

# Costos Unitarios de Transferencia (en €/MilGb)
Costo_Unitario_Centimos = {
    ('Lisboa', 'Paris'): 4, ('Lisboa', 'Berlin'): 3, ('Lisboa', 'Varsovia'): 6,
    ('Madrid', 'Paris'): 7, ('Madrid', 'Berlin'): 4, ('Madrid', 'Varsovia'): 9,
    ('Turin', 'Paris'): 9, ('Turin', 'Berlin'): 5, ('Turin', 'Varsovia'): 2
}
Costo_Unitario = {k: v / 100 for k, v in Costo_Unitario_Centimos.items()}

# Costo variable para rutas vía Zúrich (5 céntimos)
Costo_Variable_Transbordo = 0.05

# Parámetros de Costos Fijos y Capacidad
COSTO_FIJO_ORIGEN = 5000.0
COSTO_FIJO_CANAL_BASE = 50.0
CAPACIDAD_CANAL = 10.0
MAX_CANALES_BASE = 4

# Parámetros del Canal Adicional
COSTO_FIJO_ADICIONAL = 65.0
MIN_FLUJO_ZP = 0.5  # Flujo mínimo si se activa Zurich->París

In [3]:
# --- 2. CREACIÓN DEL MODELO ---
m = gp.Model("TransporteTransbordo")

Set parameter Username
Set parameter LicenseID to value 2718415
Academic license - for non-commercial use only - expires 2026-10-06


In [4]:
# --- 3. VARIABLES DE DECISIÓN ---

# Flujo: Rutas directas, a T, y desde T (Continuas)
x = m.addVars(Todas_Rutas, vtype=GRB.CONTINUOUS, name="Flujo")

# Número de Canales: Rutas directas, a T, y desde T (Enteras)
y = m.addVars(Todas_Rutas, vtype=GRB.INTEGER, name="NumCanales")

# Activación de Origen (Binaria)
a = m.addVars(Origenes, vtype=GRB.BINARY, name="ActivacionOrigen")

# Alquiler de Canal Adicional (Binaria)
z = m.addVar(vtype=GRB.BINARY, name="CanalAdicional")

In [5]:
# --- 4. FUNCIÓN OBJETIVO ---

# Flujo variable rutas directas
Costo_Var_Directas = gp.quicksum(Costo_Unitario[i, j] * x[i, j] for i, j in Rutas_Directas)
# Flujo variable rutas vía Zúrich
Costo_Var_Transbordo = Costo_Variable_Transbordo * (x.sum(Origenes, Transbordo) + x.sum(Transbordo, Destinos))
Costo_Variable_Total = Costo_Var_Directas + Costo_Var_Transbordo

# Costos Fijos: Canales
Costo_Fijo_Canales = COSTO_FIJO_CANAL_BASE * y.sum()
Costo_Fijo_Adicional = COSTO_FIJO_ADICIONAL * z

# Costos Fijos: Orígenes
Costo_Fijo_Origenes = COSTO_FIJO_ORIGEN * a.sum()

m.setObjective(Costo_Variable_Total + Costo_Fijo_Canales + Costo_Fijo_Adicional + Costo_Fijo_Origenes, GRB.MINIMIZE)

In [6]:
# --- 5. RESTRICCIONES ---

# A. Restricciones de Balance de Flujo

# A1. Oferta (Origen i): Flujo total saliente (directo + a Zúrich) <= Oferta S_i
m.addConstrs((x.sum(i, '*') <= Oferta[i] for i in Origenes), name="Oferta")

# A2. Demanda (Destino j): Flujo total entrante (directo + desde Zúrich) = Demanda D_j
m.addConstrs((x.sum('*', j) == Demanda[j] for j in Destinos), name="Demanda")

# A3. Balance en Zúrich (T): Flujo entrante = Flujo saliente
m.addConstr(x.sum(Origenes, Transbordo) == x.sum(Transbordo, Destinos), name="Balance_Zurich")

# B. Restricciones de Enlace Lógico y Capacidad

# B1. Enlace Capacidad: Flujo <= Capacidad total de canales (x_ij <= M * y_ij)
m.addConstrs((x[i, j] <= CAPACIDAD_CANAL * y[i, j] for i, j in Todas_Rutas), name="Capacidad_Canal")

# B2. Enlace Origen/Activación: Flujo saliente total <= M * a[i]
# Usamos la Oferta[i] como M
m.addConstrs((x.sum(i, '*') <= Oferta[i] * a[i] for i in Origenes), name="Enlace_Origen_Activacion")

# C. Restricciones Globales y Condicionales

# C1. Número Máximo de Canales en la Red (Sumatoria de todos los y_ij)
m.addConstr(y.sum() <= MAX_CANALES_BASE + z, name="MaximoCanalesRed")

# C2. Exclusión Mutua (Berlín)
m.addConstr(y['Lisboa', 'Berlin'] + y['Madrid', 'Berlin'] <= 1, name="ExclusionMutua_Berlin")

# C3. Condición de Mínimo Flujo (Zúrich -> París)
# Si y['Zurich', 'Paris'] >= 1, entonces x['Zurich', 'Paris'] >= 0.5
m.addConstr(x[Transbordo, 'Paris'] >= MIN_FLUJO_ZP * y[Transbordo, 'Paris'], name="MinFlujo_ZP")

<gurobi.Constr *Awaiting Model Update*>

In [7]:
# --- 6. OPTIMIZAR Y RESOLVER ---
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 28 rows, 34 columns and 95 nonzeros
Model fingerprint: 0x52a0abf5
Variable types: 15 continuous, 19 integer (4 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+01]
  Objective range  [2e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 25 rows, 34 columns, 83 nonzeros
Variable types: 15 continuous, 19 integer (19 binary)
Found heuristic solution: objective 15316.460000
Found heuristic solution: objective 15316.230000

Root relaxation: objective 1.516734e+04, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

In [8]:
# --- 7. IMPRIMIR RESULTADOS (BLOQUE CORREGIDO) ---
if m.status == GRB.OPTIMAL:
    print("\n" + "=" * 60)
    print("SOLUCIÓN ÓPTIMA ENCONTRADA (Red con Transbordo)")
    print(f"Costo Total Mínimo: {m.objVal:.2f} €")
    print("=" * 60)

    # Orígenes Activados
    origenes_act = [i for i in Origenes if a[i].x > 0.5]
    print(f"\n--- 💸 COSTO FIJO DE ORÍGENES ({COSTO_FIJO_ORIGEN} € c/u) ---")
    print(f"Orígenes Activados: {', '.join(origenes_act)}")

    # Flujos y Canales
    canales_totales = 0
    # CORRECCIÓN: Usar .getValue() en la expresión lineal x.sum(...)
    flujo_Z_in = x.sum(Origenes, Transbordo).getValue()
    flujo_Z_out = x.sum(Transbordo, Destinos).getValue()

    print(f"\n--- DETALLE DE FLUJOS Y CANALES ---")
    for i, j in Todas_Rutas:
        # Aquí se usa .x correctamente porque y[i, j] es una variable individual
        num_canales = int(y[i, j].x)
        if num_canales > 0:
            canales_totales += num_canales
            # Aquí se usa .x correctamente porque x[i, j] es una variable individual
            print(
                f"[{i} -> {j}]: Flujo = {x[i, j].x:.2f} MilGb | Canales = {num_canales} (Cap. {num_canales * CAPACIDAD_CANAL:.0f} MilGb)")

    print(f"\nFlujo en Zúrich (Entrada/Salida): {flujo_Z_in:.2f} MilGb")

    # Resumen de Costos Fijos
    costo_fijo_origen_final = len(origenes_act) * COSTO_FIJO_ORIGEN
    costo_fijo_canales_final = canales_totales * COSTO_FIJO_CANAL_BASE
    costo_fijo_adicional_final = z.x * COSTO_FIJO_ADICIONAL

    print("\n--- RESUMEN DE COSTOS TOTALES ---")

    # CORRECCIÓN: Usar .getValue() en la expresión lineal Costo_Variable_Total
    print(f"  Costo Variable: {Costo_Variable_Total.getValue():.2f} €")
    print(f"  Costo Fijo Orígenes: {costo_fijo_origen_final:.2f} €")
    print(f"  Costo Fijo Canales Base: {costo_fijo_canales_final:.2f} € ({canales_totales} canales)")
    print(f"  Canal Adicional Alquilado (z): {'SÍ' if z.x > 0.5 else 'NO'} ({costo_fijo_adicional_final:.2f} €)")

    print(f"  Total: {m.objVal:.2f} €")

elif m.status == GRB.INFEASIBLE:
    print("El modelo no tiene solución factible.")


SOLUCIÓN ÓPTIMA ENCONTRADA (Red con Transbordo)
Costo Total Mínimo: 15200.75 €

--- 💸 COSTO FIJO DE ORÍGENES (5000.0 € c/u) ---
Orígenes Activados: Lisboa, Madrid, Turin

--- DETALLE DE FLUJOS Y CANALES ---
[Lisboa -> Berlin]: Flujo = 5.00 MilGb | Canales = 1 (Cap. 10 MilGb)
[Madrid -> Paris]: Flujo = 4.00 MilGb | Canales = 1 (Cap. 10 MilGb)
[Madrid -> Varsovia]: Flujo = 2.00 MilGb | Canales = 1 (Cap. 10 MilGb)
[Turin -> Varsovia]: Flujo = 7.00 MilGb | Canales = 1 (Cap. 10 MilGb)

Flujo en Zúrich (Entrada/Salida): 0.00 MilGb

--- RESUMEN DE COSTOS TOTALES ---
  Costo Variable: 0.75 €
  Costo Fijo Orígenes: 15000.00 €
  Costo Fijo Canales Base: 200.00 € (4 canales)
  Canal Adicional Alquilado (z): NO (-0.00 €)
  Total: 15200.75 €
